In [0]:
!wget -O quora.zip -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1ERtxpdWOgGQ3HOigqAMHTJjmOE_tWvoF"
!unzip quora.zip
!pip install -q --upgrade nltk gensim bokeh pandas

import nltk
nltk.download('punkt')
nltk.download('stopwords')

# Word embeddings

The notebook is largely based on [the first assignment from the relevant SHAD course](https://github.com/yandexdataschool/nlp_course/tree/master/week1_embeddings).*

Everyone saw these pictures (I hope):
![embeddings relations](https://www.tensorflow.org/images/linear-relationships.png)
*From [Vector Representations of Words, Tensorflow tutorial](https://www.tensorflow.org/tutorials/representation/word2vec)*

Today we will deal with such models.

Let's start the morning with visualizations. We go to the site [http://rusvectores.org/ru/ru/ (http: //rusvectores.org/ru/) and see what the trained models for Russian can do.

Note the sections * Related words * and * Calculator *, as well as the set of models that you can choose from.

## We train a simple model

t’s just not fun to just look at other people's models, so we’ll gradually come closer to solving a specific task: [Quora Question Pairs at kaggle](https://www.kaggle.com/c/quora-question-pairs):

In [0]:
import pandas as pd

quora_data = pd.read_csv('train.csv')

quora_data

Поучим на этих текстах Word2vec из gensim. 

Для начала объединим все тексты.

In [0]:
import numpy as np

quora_data.question1 = quora_data.question1.replace(np.nan, '', regex=True)
quora_data.question2 = quora_data.question2.replace(np.nan, '', regex=True)

texts = list(pd.concat([quora_data.question1, quora_data.question2]).unique())
texts[:10]

Для токенизации проще всего воспользоваться `nltk` (он быстрее `spacy`, но может быть хуже в отдельных случаях).

In [0]:
from nltk.tokenize import word_tokenize

word_tokenize(texts[0])

**Задание** Приведите все тексты к нижнему регистру и токенизируйте их.

In [0]:
tokenized_texts = <do smth>

assert all(isinstance(row, (list, tuple)) for row in tokenized_texts), \
    "please convert each line into a list of tokens"
assert all(all(isinstance(tok, str) for tok in row) for row in tokenized_texts), \
    "please convert each line into a list of tokens"

is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(not is_latin(token) or token.islower() for tokens in tokenized_texts for token in tokens),\
    "please lowercase each line"

In [0]:
print([' '.join(row) for row in tokenized_texts[:2]])

Потренируем небольшую модель на полученных текстах:

In [0]:
from gensim.models import Word2Vec

model = Word2Vec(tokenized_texts, 
                 size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

## Изучаем полученную модель

Ура, теперь можно делать то же, что было на `rusvectores`.

Получить вектор для слова:

In [0]:
model.get_vector('anything')

Найти наиболее близкие слова:

In [0]:
model.most_similar('bread')

Или даже так:

In [0]:
model.most_similar(positive=['coder', 'money'], negative=['brain'])

И так, конечно:

In [0]:
model.most_similar([model.get_vector('politician') - model.get_vector('power') + model.get_vector('honesty')])

**Задание** Поищите аналогии самостоятельно. 

## Визуализируем модель

Посмотрим на проекции первых тысячи самых частотных слов.

In [0]:
words = sorted(model.vocab.keys(), 
               key=lambda word: model.vocab[word].count,
               reverse=True)[:1000]

print(words[::100])

**Задание** Постройте матрицу из эмбеддингов этих слов:

In [0]:
word_vectors = model.vectors[[model.vocab[word].index for word in words]]

assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), model.vectors.shape[1])
assert np.isfinite(word_vectors).all()

### PCA

The simplest linear dimension reduction method is __P__rincipial __C__omponent __A__nalysis.

The PCA searches for axes on which the data will have the largest scatter when projected.

![pca](https://i.stack.imgur.com/Q7HIP.gif)
*From [https://stats.stackexchange.com/a/140579](https://stats.stackexchange.com/a/140579)*

As a result, you can take the projections onto the first few components - and save as much information as possible, reducing the dimension.

Beautiful visualizations can be found [here] (http://setosa.io/ev/principal-component-analysis/).

** Task ** Use the PCA from sklearn, and then center and normalize the result.

In [0]:
<imports>

def get_pca_projection(word_vectors):
    <code>

In [0]:
word_vectors_pca = get_pca_projection(word_vectors)

assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1 - word_vectors_pca.std(0))) < 1e-5, "points must have unit variance"

Visualize what happened:

In [0]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    output_notebook()
    
    if isinstance(color, str): 
        color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: 
        pl.show(fig)
    return fig

In [0]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

### TSNE

A more interesting and complex (non-linear) method for visualizing high-dimensional spaces is TSNE. You can look at it in detail [here] (https://distill.pub/2016/misread-tsne/) (even more beautiful pictures!).

** Task ** As with PCA, use TSNE from sklearn and normalize + center the result.

In [0]:
from sklearn.manifold import TSNE

def get_tsne_projection(word_vectors):
    <your code>

In [0]:
word_tsne = get_tsne_projection(word_vectors)
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)

## Phrased phrases

Now for tests it will be useful to use fixed embeddings. Let's load already trained model.
(see which models and models are available at all, by calling `api.info ()`)

In [0]:
import gensim.downloader as api

model = api.load('glove-twitter-100')

A simple and cheap way to calculate embedding of a phrase, when there are embeddingings of words - to average.

So let's do this: tokenize and lower-case the phrase, average the embeddingings of the words for which they are counted.

**Task** Write a function to count phrase embedding.

In [0]:
def get_phrase_embedding(model, phrase):    
    vector = np.zeros([model.vector_size], dtype='float32')
    
    <write me>
    
    return vector

In [0]:
vector = get_phrase_embedding(model, "I'm very sure. This never happened to me before...")

assert np.allclose(vector[::10],
                   np.array([ 0.30757686, -0.05861897,  0.143751  , -0.11104885, -0.96929336,
                             -0.21928601,  0.21652265,  0.14978765,  1.4842536 ,  0.017826  ],
                              dtype=np.float32))


Calculate the vector of all phrases.

In [0]:
text_vectors = np.array([get_phrase_embedding(model, phrase) for phrase in tokenized_texts])

And learn to look for coming!

**Assignment** Find the closest questions to it by line.

In [0]:
from sklearn.metrics.pairwise import cosine_similarity

def find_nearest(model, text_vectors, texts, query, k=10):
    <write me, please>

In [0]:
results = find_nearest(model, text_vectors, texts, query="How do i enter the matrix?", k=10)

print('\n'.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[1] == 'How do I get to the dark web?'
assert results[4] == 'What can I do to save the world?'

In [0]:
find_nearest(model, text_vectors, texts, query="How does Trump?", k=10)

In [0]:
find_nearest(model, text_vectors, texts, query="Why don't i ask a question myself?", k=10)

## Begin the classification

### Bag-of-words

**Task** Let's collect tokens for the beginning.

In [0]:
tokenized_question1 = ...
tokenized_question2 = ...

**Task** Calculate cosine proximity between questions.

The easiest way is to implement the function of its calculation by hand:

$$\text{cosine_similarity}(x, y) = \frac{x^{T} y}{||x||\cdot ||y||}$$

In [0]:
<do smth>

cosine_similarities = <and calc the result>

We select the proximity threshold at which we will consider the texts to be the same.

**Task** Implement the `accuracy` function that calculates the accuracy of the classification at a given threshold.

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

def accuracy(cosine_similarities, threshold):
    return <accuracy>

thresholds = np.linspace(0, 1, 100, endpoint=False)
plt.plot(thresholds, [accuracy(cosine_similarities, th) for th in thresholds])

And run the optimization:

In [0]:
from scipy.optimize import minimize_scalar

res = minimize_scalar(lambda x: -accuracy(cosine_similarities, x), bounds=(0.5, 0.99), method='bounded')
best_threshold = res.x
print('Threshold = {:.5f}, Accuracy = {:.2%}'.format(best_threshold, accuracy(cosine_similarities, best_threshold)))

### Tf-idf weights

Instead of stupidly averaging the vectors, you can average them based on the weights - the familiar tf-idf will help in this.

** Task ** Calculate weighted vector of questions. Use `TfidfVectorizer`

`TfidfVectorizer` returns the matrix` (samples_count, words_count) `. And our embeddings have the dimension `(words_count, embedding_dim)`. So you can just multiply them. Then each phrase - the sequence of words $ w_1, \ ldots, w_k $ - is transformed into the vector $ \ sum_i \ text {idf} (w_i) \ cdot \ text {embedding} (w_i) $. This vector should probably be normalized to the number of words $ k $.

** Task ** In addition to tf-idf, you can add filtering of stop words and punctuation.
Stop words can be taken from `nltk`:

```python
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
```

In [0]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from string import punctuation

<calculate tf-idf weighted vectors>

Посмотрим, что получается после фильтрации и векторизации:

In [0]:
for col in tfidf_question1[0].tocoo().col:
    print(model.index2word[col], end=' ')

print('\n' + ' '.join(tokenized_question1[0]))

**Task** Calculate quality with new vectors.

In [0]:
cosine_similarities = <calc it somehow>

res = minimize_scalar(lambda x: -accuracy(cosine_similarities, x), bounds=(0.8, 0.99), method='bounded')
best_threshold = res.x
print('Threshold = {:.5f}, Accuracy = {:.2%}'.format(best_threshold, accuracy(cosine_similarities, best_threshold)))

## Let's look inside learning word embeddings

The key idea is that a word can be defined by the context in which it occurs:
![contexts](https://image.ibb.co/mnQ2uz/2018_09_17_21_07_08.png)
*From [cs224n, Lecture 2](http://web.stanford.edu/class/cs224n/lectures/lecture2.pdf)*

Watch how everything learns, we will be here: [https://ronxin.github.io/wevi/](https://ronxin.github.io/wevi/).

## Got a word by word machine translation!

In [0]:
!wget -O ukr_rus.train.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1vAK0SWXUqei4zTimMvIhH3ufGPsbnC_O"
!wget -O ukr_rus.test.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1W9R2F8OeKHXruo2sicZ6FgBJUTJc8Us_"
!wget -O fairy_tale.txt -qq --no-check-certificate "https://drive.google.com/uc?export=download&id=1sq8zSroFeg_afw-60OmY8RATdu_T1tej"

# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id': '1d7OXuil646jUeDS1JNhP9XWlZogv6rbu'})
downloaded.GetContentFile('cc.ru.300.vec.zip')

downloaded = drive.CreateFile({'id': '1yAqwqgUHtMSfGS99WLGe5unSCyIXfIxi'})
downloaded.GetContentFile('cc.uk.300.vec.zip')

!unzip cc.ru.300.vec.zip
!unzip cc.uk.300.vec.zip


Let's write a simple implementation of the machine translation model.

The idea is based on the article [Word Translation Without Parallel Data] (https://arxiv.org/pdf/1710.04087.pdf). The authors in the repository have a lot more interesting: [https://github.com/facebookresearch/MUSE_ ((https://github.com/facebookresearch/MUSE)

And we will translate from Ukrainian to Russian.

! [] (https://raw.githubusercontent.com/yandexdataschool/nlp_course/master/resources/blue_cat_blue_whale.png)

*blue kit* vs. *blue kit*

In [0]:
from gensim.models import KeyedVectors

ru_emb = KeyedVectors.load_word2vec_format("cc.ru.300.vec")
uk_emb = KeyedVectors.load_word2vec_format("cc.uk.300.vec")


Look at a couple of serpeni-august (being a translation)

In [0]:
ru_emb.most_similar([ru_emb["август"]])

In [0]:
uk_emb.most_similar([uk_emb["серпень"]])

In [0]:
ru_emb.most_similar([uk_emb["серпень"]])

In [0]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r", encoding='utf8') as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)


uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

### Учим маппинг из одного пространства эмбеддингов в другое

У нас есть пары слов, соответствующих друг другу, и их эмбеддинги. Найдем преобразование из одного пространства в другое, чтобы приблизить известные нам слова:

$$W^*= \arg\min_W ||WX - Y||_F, \text{где} ||*||_F - \text{норма Фробениуса}$$

Эта функция очень похожа на линейную регрессию (без биаса).

**Задание** Реализуйте её - воспользуйтесь `LinearRegression` из sklearn с `fit_intercept=False`:

In [0]:
mapping = ...

Проверим, куда перейдет `серпень`:

In [0]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)

Должно получиться, что в топе содержатся разные месяцы, но август не первый.

Будем мерять percision top-k с k = 1, 5, 10.

**Задание** Реализуйте следующую функцию:

In [0]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(ru_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        <write code here>
    precision_val = num_matches / len(pairs)
    return precision_val

In [0]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [0]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [0]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

assert precision_top1 >= 0.635
assert precision_top5 >= 0.813

### Улучшаем маппинг

Можно показать, что маппинг лучше строить ортогональным:
$$W^*= \arg\min_W ||WX - Y||_F \text{, где: } W^TW = I$$

Искать его можно через SVD:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$

$$W^*=UV^T$$

**Задание** Реализуйте эту функцию.

In [0]:
def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    <write code there>

In [0]:
W = learn_transform(X_train, Y_train)

In [0]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

In [0]:
assert precision(uk_ru_test, np.matmul(X_test, W)) >= 0.653
assert precision(uk_ru_test, np.matmul(X_test, W), 5) >= 0.824

### Пишем переводчик


We implement a simple word-for-word translator - for each word we will look for its nearest neighbor in the common embedding space. If the word is not in the inserts - just copy it.

In [0]:
with open("fairy_tale.txt", "r") as in f:
    uk_sentences = [line.rstrip().lower() for line in in f]

In [0]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    <implement it!>

In [0]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

In [0]:
for sentence in uk_sentences:
    print("src: {}\ndst: {}\n".format(sentence, translate(sentence)))

## Referrence

[On word embeddings - Part 1, Sebastian Ruder](http://ruder.io/word-embeddings-1/)  
[Deep Learning, NLP, and Representations, Christopher Olah](http://colah.github.io/posts/2014-07-NLP-RNNs-Representations/)  
[Making Sense of Word Embeddings (2016), Pelevina et al](http://anthology.aclweb.org/W16-1620)    
[Evaluation methods for unsupervised word embeddings (2015), T. Schnabel](http://www.aclweb.org/anthology/D15-1036)  
[Intrinsic Evaluation of Word Vectors Fails to Predict Extrinsic Performance (2016), B. Chiu](https://www.aclweb.org/anthology/W/W16/W16-2501.pdf)  
[Problems With Evaluation of Word Embeddings Using Word Similarity Tasks (2016), M. Faruqui](https://arxiv.org/pdf/1605.02276.pdf)  
[Improving Reliability of Word Similarity Evaluation by Redesigning Annotation Task and Performance Measure (2016), Oded Avraham, Yoav Goldberg](https://arxiv.org/pdf/1611.03641.pdf)  
[Evaluating Word Embeddings Using a Representative Suite of Practical Tasks (2016), N. Nayak](https://cs.stanford.edu/~angeli/papers/2016-acl-veceval.pdf)  
[Word Vector Representations: word2vec, Lecture 2, cs224n](https://www.youtube.com/watch?v=ERibwqs9p38)